Performing EDA on this bike sharing demand dataset and trying to apply linear regression. Still a work in progress -  need to check for outliers, perform outlier removal and then resume.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(os.listdir("../input/bike-sharing-demand/"))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading the dataset
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#Exploratory data analysis
#Checking for nulls in train
print("Checking for Nulls in Training set")
print(train.isnull().sum().sort_values(ascending = False))
print("Checking for Nulls in Test set")
print(test.isnull().sum().sort_values(ascending = False))


In [ ]:
#understanding the dataframe for train and test
print("Checking for Nulls in Training set")
print(train.info())
print("Checking for Nulls in Test set")
print(test.info())

Since the datatype of the 'datetime' column is an object, it has to be converted to 'datetime' , to perform the date related operations.

In [ ]:
#Changing the type for column 'datetime' to datetime. This is to help perform subsequent date and time operations
train.datetime = pd.to_datetime(train.datetime)
test.datetime = pd.to_datetime(test.datetime)

In [ ]:
print("Checking for Nulls in Training set")
print(train.info())
print("Checking for Nulls in Test set")
print(test.info())

Now let's derive additional attributes from the datetime column

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['dayofweek'] = train['datetime'].dt.day_name()
train['hour'] = train['datetime'].dt.hour



test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['dayofweek'] = test['datetime'].dt.day_name()
test['hour'] = test['datetime'].dt.hour


Now that the basic features have been set up, we can proceed with exploratory analysis of those features. Starting off with profiling the 'Count' column

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(train['count'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(train["count"][0:800])
plt.show()

Now, we can compare 'season' plays a role in determining the 'count'

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='season', y='count', data=train)

Thus, we see that people rent more bikes in seasons 2 and 3, which are Summer and Fall. Winter demand is more than Spring. 

Now proceeding further to check the demand against the day of the week and the hour of the day. 

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='dayofweek', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='hour', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='holiday', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='workingday', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='weather', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='year', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='dayofweek', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='casual', y='count', data=train)

plt.figure(figsize=(10,5))
sns.boxplot(x='registered', y='count', data=train)

Performing graphical analysis on the continuous features

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1,)
plt.title('Temperature Vs Count')
plt.xlabel('Temperature')
plt.ylabel('Count')
plt.scatter(train['temp'],train['count'],s=2,c='b')

plt.subplot(2,2,2)
plt.title('Abs Temperature Vs Count')
plt.xlabel('Abs Temperature')
plt.ylabel('Count')
plt.scatter(train['atemp'],train['count'],s=2,c='g')

plt.subplot(2,2,3)
plt.title('Humidity Vs Count')
plt.xlabel('Humidity')
plt.ylabel('Count')
plt.scatter(train['humidity'],train['count'],c='r')

plt.subplot(2,2,4)
plt.title('Windspeed Vs Count')
plt.xlabel('Windspeed')
plt.ylabel('Count')
plt.scatter(train['windspeed'],train['count'],c='c')

plt.tight_layout()

Checking if there are any outliers

In [ ]:
train_dup = train.copy()
train_dup['count'].describe()

In [ ]:
train_dup['count'].quantile([0.1,0.2,0.25,0.8,0.9,0.99])

In [ ]:
multicol_check = train_dup[['temp','atemp','humidity','count']].corr()
multicol_check

In [ ]:
#Dropping the unwanted columns

train_dup = train_dup.drop(['atemp','holiday','workingday','year','dayofweek'],axis=1)
train_dup.head()

In [ ]:
train_dup['season'] = train_dup['season'].astype('category')
train_dup['weather'] = train_dup['weather'].astype('category')
train_dup['month'] = train_dup['month'].astype('category')
train_dup['hour'] = train_dup['hour'].astype('category')

print(train_dup.info())


In [ ]:
#Now, to train the model
from sklearn.model_selection import train_test_split
X = train_dup[['season','weather','temp','humidity','windspeed','casual','registered','month','day','hour']]
y = train_dup['count']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)


In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LinearRegression 

std_reg = LinearRegression()
std_reg.fit(X_train,y_train)

In [ ]:
r2_train = std_reg.score(X_train,y_train)
r2_test = std_reg.score(X_test,y_test)
print('R Squared Error for Train set:',r2_train)
print('R Squared Error for Test set:',r2_test)